# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data and Setup

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

In [3]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


### Polynomial Conversion

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [6]:
poly_features = polynomial_converter.fit_transform(X)

In [7]:
poly_features.shape

(200, 19)

In [10]:
poly_features

array([[2.30100000e+02, 3.78000000e+01, 6.92000000e+01, ...,
        9.88757280e+04, 1.81010592e+05, 3.31373888e+05],
       [4.45000000e+01, 3.93000000e+01, 4.51000000e+01, ...,
        6.96564990e+04, 7.99365930e+04, 9.17338510e+04],
       [1.72000000e+01, 4.59000000e+01, 6.93000000e+01, ...,
        1.46001933e+05, 2.20434291e+05, 3.32812557e+05],
       ...,
       [1.77000000e+02, 9.30000000e+00, 6.40000000e+00, ...,
        5.53536000e+02, 3.80928000e+02, 2.62144000e+02],
       [2.83600000e+02, 4.20000000e+01, 6.62000000e+01, ...,
        1.16776800e+05, 1.84062480e+05, 2.90117528e+05],
       [2.32100000e+02, 8.60000000e+00, 8.70000000e+00, ...,
        6.43452000e+02, 6.50934000e+02, 6.58503000e+02]])

In [8]:
polynomial_converter.get_feature_names_out()

array(['TV', 'radio', 'newspaper', 'TV^2', 'TV radio', 'TV newspaper',
       'radio^2', 'radio newspaper', 'newspaper^2', 'TV^3', 'TV^2 radio',
       'TV^2 newspaper', 'TV radio^2', 'TV radio newspaper',
       'TV newspaper^2', 'radio^3', 'radio^2 newspaper',
       'radio newspaper^2', 'newspaper^3'], dtype=object)

In [9]:
poly_feature_names = list(polynomial_converter.get_feature_names_out())
poly_features_df = pd.DataFrame(poly_features,columns=poly_feature_names)
poly_features_df.head()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
0,230.1,37.8,69.2,52946.01,8697.78,15922.92,1428.84,2615.76,4788.64,1.218288e+07,2001359.178,3663863.892,328776.084,601886.376,1101866.064,54010.152,98875.728,181010.592,331373.888
1,44.5,39.3,45.1,1980.25,1748.85,2006.95,1544.49,1772.43,2034.01,8.812112e+04,77823.825,89309.275,68729.805,78873.135,90513.445,60698.457,69656.499,79936.593,91733.851
2,17.2,45.9,69.3,295.84,789.48,1191.96,2106.81,3180.87,4802.49,5.088448e+03,13579.056,20501.712,36237.132,54710.964,82602.828,96702.579,146001.933,220434.291,332812.557
3,151.5,41.3,58.5,22952.25,6256.95,8862.75,1705.69,2416.05,3422.25,3.477266e+06,947927.925,1342706.625,258412.035,366031.575,518470.875,70444.997,99782.865,141338.925,200201.625
4,180.8,10.8,58.4,32688.64,1952.64,10558.72,116.64,630.72,3410.56,5.910106e+06,353037.312,1909016.576,21088.512,114034.176,616629.248,1259.712,6811.776,36834.048,199176.704


In [15]:
poly_features_df.describe()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.00000,2.000000e+02,2.000000e+02,2.000000e+02,200.000000,2.000000e+02,2.000000e+02,200.000000,200.000000,200.000000,2.000000e+02
mean,147.042500,23.264000,30.554000,28955.591950,3490.309900,4598.126400,760.539300,824.73275,1405.48370,6.371006e+06,7.033242e+05,9.270578e+05,115027.471030,1.249737e+05,2.155203e+05,28201.560860,29000.037585,42395.923715,8.088636e+04
std,85.854236,14.846809,21.778621,25565.429396,3360.740127,4870.716495,735.807704,937.81978,1863.38301,7.082500e+06,8.716341e+05,1.291333e+06,147815.336657,1.841252e+05,3.638612e+05,34214.831852,40818.085389,71471.522336,1.664323e+05
min,0.700000,0.000000,0.300000,0.490000,0.000000,6.090000,0.000000,0.00000,0.09000,3.430000e-01,0.000000e+00,4.263000e+00,0.000000,0.000000e+00,8.600000e+00,0.000000,0.000000,0.000000,2.700000e-02
25%,74.375000,9.975000,12.750000,5531.957500,773.445000,1119.747500,99.502500,147.93750,162.57000,4.114863e+05,6.072033e+04,8.421447e+04,7658.357000,1.461359e+04,1.575378e+04,992.574750,1154.578500,1602.270000,2.072958e+03
50%,149.750000,22.900000,25.750000,22425.065000,2069.065000,2809.675000,524.570000,416.11500,663.08500,3.358154e+06,2.822733e+05,3.975226e+05,52221.407500,4.500682e+04,8.082070e+04,12019.981000,8395.255000,9585.550000,1.707560e+04
75%,218.825000,36.525000,45.100000,47884.697500,5516.197500,6492.930000,1334.077500,1243.19000,2034.01000,1.047851e+07,1.095034e+06,1.311138e+06,182535.391000,1.621647e+05,2.404681e+05,48727.317750,42124.215000,53391.312000,9.173385e+04
max,296.400000,49.600000,114.000000,87852.960000,13540.410000,29906.760000,2460.160000,4172.40000,12996.00000,2.603962e+07,3.749340e+06,8.864364e+06,662126.049000,1.085615e+06,3.017592e+06,122023.936000,179340.750000,475653.600000,1.481544e+06


### Train | Test Split

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# X_train, X_test, y_train, y_test = train_test_split(
#    poly_features, y, test_size=0.3, random_state=101)
## Or 
X_train, X_test, y_train, y_test = train_test_split(
    poly_features_df, y, test_size=0.3, random_state=101)

----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
# help(StandardScaler)

In [18]:
scaler = StandardScaler() # mean is 0 and std is 1

In [27]:
scaler.fit(X_train)  # Fit only with training data

StandardScaler()

In [20]:
X_train = scaler.transform(X_train)

In [21]:
X_test = scaler.transform(X_test)

In [22]:
X_train.mean()

np.float64(7.47939721852737e-17)

In [23]:
X_train.std()

np.float64(1.0)

In [24]:
X_test.mean()

np.float64(-0.09036461660617744)

In [25]:
X_test.std()

np.float64(1.0155973297593028)

## Ridge Regression

Make sure to view video lectures for full explanation of Ridge Regression and choosing an alpha.

In [28]:
from sklearn.linear_model import Ridge

In [29]:
ridge_model = Ridge(alpha=10)

In [30]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [31]:
test_predictions = ridge_model.predict(X_test)

In [32]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [33]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [34]:
MAE

np.float64(4.053037953173393)

In [35]:
RMSE

np.float64(4.71966062959796)

How did it perform on the training set? (This will be used later on for comparison)

In [36]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

np.float64(3.7146227240273006)

### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [37]:
from sklearn.linear_model import RidgeCV

In [38]:
# help(RidgeCV)

In [40]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [41]:
# The more alpha options you pass, the longer this will take.
# Fortunately our data set is still pretty small
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [42]:
ridge_cv_model.alpha_

np.float64(0.1)

In [43]:
test_predictions = ridge_cv_model.predict(X_test)

In [44]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [45]:
MAE

np.float64(3.998839785951385)

In [46]:
RMSE

np.float64(4.553884518479703)

In [47]:
# Training Set Performance
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

np.float64(3.431316238562951)

In [48]:
ridge_cv_model.coef_

array([18.70217413,  8.48745568, -0.19210744])

In [ ]:
# ridge_cv_model.feature_names_in_

In [49]:
ridge_cv_model.intercept_

np.float64(-3.7679826014486917)


-----

## Lasso Regression

In [50]:
from sklearn.linear_model import LassoCV

In [52]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [53]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [54]:
lasso_cv_model.alpha_

np.float64(0.05505401487951766)

In [55]:
test_predictions = lasso_cv_model.predict(X_test)

In [56]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [57]:
MAE

np.float64(3.8709999422300085)

In [58]:
RMSE

np.float64(4.497871724347282)

In [59]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

np.float64(3.4867981911833397)

In [60]:
lasso_cv_model.coef_

array([12.64916865,  2.58393867, -1.4166954 ])

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [61]:
from sklearn.linear_model import ElasticNetCV

In [62]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [63]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [64]:
elastic_model.l1_ratio_

np.float64(0.99)

In [65]:
test_predictions = elastic_model.predict(X_test)

In [66]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [67]:
MAE

np.float64(3.9739748195282583)

In [68]:
RMSE

np.float64(4.532855110447627)

In [69]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

np.float64(3.4305312725164683)

In [70]:
elastic_model.coef_

array([18.26297058,  7.81640667, -0.        ])

# Normalization

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

# Loading Data
df = pd.read_csv("Advertising.csv")

# Preparing input (X) and output (y)
X = df.drop('sales',axis=1)
y = df['sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)

print(np.array(X_train).min())
print(np.array(X_train).max())

print(np.array(X_test).min())
print(np.array(X_test).max())

# Applying Normalizer
normalizer = Normalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

print("-----------")
print(X_train.min())
print(X_train.max())

print(X_test.min())
print(X_test.max())


0.0
296.4
0.3
293.6
-----------
0.0
0.9994734743591069
0.0011263285753689348
0.9984611649875367


-----
---